In [69]:
import requests

***
***

In [81]:
class UCloudRequester:

    def __init__(self, url : str, email : str) -> None:
        self.url = url
        self.email = email

    def _predict(self, prompt : str, **params) -> dict:

        try:
            query = {
                'email': self.email,
                'prompt': prompt,
                'params': {**params}
            }
            headers = {
                'Content-Type': 'application/json'
            }
            response = requests.post(self.url, headers=headers, json=query)
            return response.json()
        
        except requests.exceptions.JSONDecodeError:
            return f"Failed to decode response as JSON: {response.text}"

        except requests.exceptions.HTTPError as e:
            return "Error: " + str(e)
        
    
    def predict(self, prompt : str, return_meta=False, **params) -> dict | str:
        result : dict = self._predict(prompt, **params)
        if return_meta:
            return result
        return result["choices"][0]["text"]
    
requester = UCloudRequester(
    url="http://127.0.0.1:8000/predict",
    email="nbth.msc@cbs.dk"
)


***
***

In [82]:
params = dict(
    max_tokens=50,
    temperature=0.5,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["\n", "###"],
    # logprobs=2,
)

In [83]:
PROMPT = "Once upon a time"

In [86]:
prediction = requester.predict(prompt=PROMPT, return_meta=True, **params)

In [ ]:
prediction

In [67]:
%%timeit -n 10 -r 10

requester.predict(prompt=PROMPT, **params)

769 ms ± 47.1 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [68]:
%%timeit -n 10 -r 10

requester.predict(f"The meaning of life is", **params)

741 ms ± 134 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)
